# Precio de automóviles usados

Imaginen ustedes que tienen un amigo que quiere vender un automóvil, pero no está muy seguro de cuánto cobrar por él. ¿Qué harías?

Este es el problema que Aditya se encontró, entonces lo que hizo fue hacer web scraping (dedscargar información de la web) para obtener información de un sitio de venta de automóviles en donde estában listadas un montón de características y el precio final al que cada uno de ellos fue vendido. 

 > ❓ ¿Cómo es que nosotros podemos ayudar?
 
Accede a el dataset en el archivo `cars.csv`, puedes ver el dataset original [aquí](https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes).

In [ ]:
import pandas as pd
import numpy as np
from ydata_profiling import ProfileReport
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
cars = pd.read_csv("cars.csv")

In [ ]:
cars.head()

 > ⁉️ Y las métricas de evaluación – en la regresión no tenemos muchas opciones, podemos usar RMSE o MSE

## Análisis Exploratorio de Datos

Antes de meternos de fondo a la etapa del modelado, vamos a generar un reporte usando la biblioteca [Pandas Profiling](https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/):

In [ ]:
profile = ProfileReport(cars, title="Raw Car Dataset Analysis", explorative=True)
profile.to_file("cars-report.html")

In [ ]:
print(os.getcwd() + "/cars-report.html")

## Elimina los duplicados

Una de las grandes advertencias provistas por nuestro reporte es que existen duplicados en el dataset, así que vamos a comenzar con eso.

 > 😉 Lee la documentación de `drop_duplicates` para ver qué hacen los distintos parámetros

In [ ]:
cars = cars.drop_duplicates(keep='first')

## Divide el dataset

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
rest, test = train_test_split(cars, test_size=0.2, shuffle=True) # 20% of 100 = 20
train, val = train_test_split(rest, test_size=0.25, shuffle=True) # 25% of 80 = 20
distributions = np.array([len(train), len(val), len(test)])

print(distributions)
print(distributions / len(cars))

## *Feature engineering*

### One-hot encode categorical variables

Necesitamos una manera de pasar de una variable categórica a números, por ejemplo, tenemos en nuestro dataset una columna llamada *"maker"*, que se traduce a la constructora del automóvil: "bnw", "ford", "audi"...

Debemos encontrar una forma de convertirlos a números que nuestro algoritmo pueda usar para entrenar nuestro modelo, a este proceso se le llama  *"encoding"* (o codificación).

 > 📹 tengo un video sobre tipos de variables: [https://www.youtube.com/watch?v=SAWsQ3QmmJE](https://www.youtube.com/watch?v=SAWsQ3QmmJE)
 
Una forma de codificar variables categóricas es usando *One-Hot Encoding*, que expandirá nuestra única columna categórica en un vector (que podemos representar en forma de columnas) de 1 y 0:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
maker_encoder = OneHotEncoder()

In [ ]:
maker_encoder.fit(train[["maker"]])
mkr = maker_encoder.transform(train[["maker"]]).todense()

print(mkr.shape)

Para revisar las categorías, podemos usar la propieddad `categories_`.

In [ ]:
df = pd.DataFrame(mkr, columns=maker_encoder.categories_, index=train[["maker"]].index)
df["actual"] = train[["maker"]]
df.sample(5)

#### 🚨 `pd.get_dummies` ?

Para hacer *machine learning* no te recomiendo usar `pd.get_dummies` porque no es robusto ante datos faltantes y no preserva el estado, por ejemplo, cuando recibimos un registro para predecir en producción:


In [ ]:
test_maker = "audi"

In [ ]:
pd.get_dummies([test_maker])

In [ ]:
maker_encoder.transform([[test_maker]]).todense()

### Feature scaling

Existen algoritmos que basan su entrenamiento en únicamente números, sin contexto alguno. Algunos de ellos tienden a otorgar mayor importancia a aquellos números cuyo valor es más grande. Una apuesta segur es escalar los valores de una característica de tal modo que todos se encuentren en la misma escala, pero preservando las distancias relativas ente ellos:

In [ ]:
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
scaler = MaxAbsScaler()

In [ ]:
scaler.fit(train[["mileage"]])
scaled = scaler.transform(train[["mileage"]])

In [ ]:
values = pd.DataFrame({"mileage": train["mileage"].values, "scaled": scaled.squeeze() })
values.sample(5)

# Artefactos

Hemos visto una diversidad de herramientas que nos sirven para transformar una de nuestras observaciones del munddo real, como el diálogo emitido por una persona o un automóvil, a un grupo de números. 

Cosas como el `OneHotEncoder`, `CountVectorizer` y `MaxAbsScaler` forman parte de este conjunto de herramientas que, una vez preparadas con `fit`, debemos preservar para poder re-usarlas en producción. Estas herramientas son conocidas como artefactos.

In [ ]:
import pickle

with open("scaler.pickle", "wb") as wb:
    pickle.dump(scaler, wb)

In [ ]:
with open("scaler.pickle", "rb") as rb:
    scaler_loaded = pickle.load(rb)

In [ ]:
scaler_loaded.transform([[400]])

# Pipelines  

A lo largo del modelado creamos un montón de *artefactos* que debemos conservar para asegurarnos de que usaremos los mismos valores, parámetros e hiperparámetros. Una alternativa sería guardar cada uno de los `OneHotEncoder`, `MinMaxScaler` y cualquier otro objeto que creamos para entrenar nuestro modelo de ML.

Otra forma de hacerlo, un poco más organizada es hacer uso de un `Pipeline` de *scikit learn*:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn import set_config

# One-Hot encode maker, transmission y fuelType
one_hot_encode = ColumnTransformer([
    (
        'one_hot_encode', # Nombre de la transformación
        OneHotEncoder(sparse_output=False), # Transformación a aplicar
        ["maker", "transmission", "fuelType"] # Columnas involucradas
    )
])

# Robust encode mileage
robust_encoding = ColumnTransformer([
    ('robust_encoding', RobustScaler(), ["mileage"])
])

# Impute and standard scale mpg and tax
impute_and_scale = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('scale', MinMaxScaler())
])

standard_scaling = ColumnTransformer([
    ('standard_scaling', impute_and_scale, ["mpg", "tax"])
])

# Just pass year and engineSize
passthrough = ColumnTransformer([('passthrough', 'passthrough', ['year', "engineSize"])])

# Ensambla todo el pipeline
pipe = Pipeline([
    (
        'features',
        FeatureUnion([
            ('one_hot_encode', one_hot_encode),
            ('robust_encoding', robust_encoding),
            ('just_passs', passthrough),
            ('scale_and_impute', standard_scaling)
        ])
    )
])

In [ ]:
from sklearn import set_config

set_config(display="diagram")
pipe

In [ ]:
pipe.fit(train)

pd.DataFrame(pipe.transform(train))

In [ ]:
pd.DataFrame(pipe.transform(test))

## Modelado

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

Creamos otro pipeline, que incluya nuestro modelo de regresión elegido

In [ ]:
predicting_pipeline = Pipeline([
    ('feature', pipe),
    ('estimator', lr)
])

In [ ]:
predicting_pipeline.fit(train, train['price'])

In [ ]:
train_pred = predicting_pipeline.predict(train)
val_pred = predicting_pipeline.predict(val)

In [ ]:
pd.DataFrame({'real':val['price'], 'predicted':val_pred})

## Evaluación de los modelos

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [ ]:
train_mse = mean_absolute_error(train['price'], train_pred)
val_mse = mean_absolute_error(val['price'], val_pred)

print(f"Entrenamiento MSE: {train_mse:2.02f}\n"
      f"Validación MSE:    {val_mse:2.02f}")

### Evaluación en los datos de prueba

In [ ]:
test_pred = predicting_pipeline.predict(test)
test_mse = mean_absolute_error(test['price'], test_pred)

print(f"Prueba MSE: {test_mse:2.02f}")

# Guarda el pipeline

In [ ]:
from joblib import dump, load
dump(predicting_pipeline, 'car-prices.model') 

## Prediciendo en nuestro propio auto

In [ ]:
saved_pipeline = load('car-prices.model')

In [ ]:
maker = "ford"
model = "focus"
year = 2020
transmission = "Manual"
mileage = 50
fuelType = "Petrol"
tax = 100
mpg = 30
engineSize = 1.5

mi_automóvil = pd.DataFrame({
    "maker": [maker], "model": [model], "year": [year], "transmission": [transmission], 
    "mileage": [mileage], "fuelType": [fuelType], "tax": [tax], "mpg": [mpg], "engineSize": [engineSize],
})

price = saved_pipeline.predict(mi_automóvil).squeeze()

print(price)

## De tarea... 

 - Creamos un modelo para todas las constructoras, ¿valdría la pena crear un modelo independiente para cada una? – inténtalo y ve si los resultados mejoran.
 - Utiliza otro modelo, tal vez [XGBoost](https://xgboost.readthedocs.io/en/stable/) para ver si te da mejores resultados
 - ¿Sabes Flask, FastAPI, o Django? pon tu modelo en una API

## Para aprender más  

 - Échale un ojo a mi video sobre [tipos de variables](https://www.youtube.com/watch?v=SAWsQ3QmmJE)
 - Conoce [cuándo escalar y cuando normalizar](https://datascience.stackexchange.com/questions/45900/when-to-use-standard-scaler-and-when-normalizer) tus datos
 - Revisa la [documentación de sklearn](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling) referente a los diferentes escaladores
 - Aprende [cuándo es válido eliminar outliers](https://statisticsbyjim.com/basics/remove-outliers/) (valores extremos)
 
